# OpenCV Beginner Tutorials Notebook

This is a collection of Python code examples for the requested OpenCV topics. You can copy these into a Jupyter Notebook (.ipynb) file for interactive execution. Each section includes a brief explanation, followed by the code in a markdown code block.

Make sure you have OpenCV installed in your Python environment. If not, install it via `pip install opencv-python`. These examples use `cv2` (the OpenCV module) and assume a basic setup with a webcam for video capture.


In [2]:
import cv2
import numpy as np
from datetime import datetime

## 1. How to Read, Write, Show Videos from Camera in OpenCV

This example demonstrates capturing video from your camera (webcam), displaying it in a window, and writing (saving) it to a file. Press 'q' to quit.


In [3]:
# Open the default camera (index 0)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Define the codec and create VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Codec for .avi file
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))  # Output file, fps, frame size

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Error: Failed to capture frame.")
        break
    
    # Write the frame to the output file
    out.write(frame)
    
    # Display the resulting frame
    cv2.imshow('Camera Feed', frame)
    
    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything when done
cap.release()
out.release()
cv2.destroyAllWindows()


## 2. Draw Geometric Shapes on Images using Python OpenCV

This example loads a blank image and draws various shapes like lines, rectangles, circles, and text using OpenCV drawing functions.


In [4]:
# Create a blank black image (height, width, channels)
image = np.zeros((512, 512, 3), np.uint8)

# Draw a line (start point, end point, color (BGR), thickness)
cv2.line(image, (0, 0), (511, 511), (255, 0, 0), 5)  # Blue line

# Draw a rectangle (top-left, bottom-right, color, thickness)
cv2.rectangle(image, (384, 0), (510, 128), (0, 255, 0), 3)  # Green rectangle

# Draw a circle (center, radius, color, thickness; -1 for filled)
cv2.circle(image, (447, 63), 63, (0, 0, 255), -1)  # Red filled circle

# Draw an ellipse (center, axes lengths, angle, start/end angles, color, thickness)
cv2.ellipse(image, (256, 256), (100, 50), 0, 0, 180, (255, 255, 0), -1)  # Cyan half-ellipse

# Draw a polygon (array of points, isClosed, color, thickness)
pts = np.array([[10, 5], [20, 30], [70, 20], [50, 10]], np.int32)
pts = pts.reshape((-1, 1, 2))
cv2.polylines(image, [pts], True, (0, 255, 255), 3)  # Yellow polygon

# Add text (text, position, font, scale, color, thickness)
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(image, 'OpenCV Shapes', (10, 500), font, 1, (255, 255, 255), 2, cv2.LINE_AA)

# Display the image
cv2.imshow('Shapes', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## 3. Setting Camera Parameters in OpenCV Python

This example shows how to set properties like frame width, height, brightness, and FPS on a camera capture object.


In [ ]:
# Open the default camera
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Set camera parameters
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)   # Width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)   # Height
cap.set(cv2.CAP_PROP_FPS, 30)             # Frames per second
cap.set(cv2.CAP_PROP_BRIGHTNESS, 150)     # Brightness (0-255)
cap.set(cv2.CAP_PROP_CONTRAST, 50)        # Contrast (0-255)

# Print current parameters to verify
print("Width:", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("Height:", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("FPS:", cap.get(cv2.CAP_PROP_FPS))
print("Brightness:", cap.get(cv2.CAP_PROP_BRIGHTNESS))
print("Contrast:", cap.get(cv2.CAP_PROP_CONTRAST))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    cv2.imshow('Adjusted Camera Feed', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Width: 1280.0
Height: 720.0
FPS: 30.0
Brightness: 0.0
Contrast: 50.0



## 4. Show Date and Time on Videos using OpenCV Python

This example captures video from the camera and overlays the current date and time on each frame using `cv2.putText` and the `datetime` module.


In [ ]:
# Open the default camera
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Get current date and time
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Put text on the frame (text, position, font, scale, color, thickness)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, current_time, (10, 50), font, 1, (0, 255, 255), 2, cv2.LINE_AA)
    
    # Display the frame
    cv2.imshow('Video with Date/Time', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 5. Handle Mouse Events in OpenCV

This example sets up a mouse callback to handle events like left-click (draw a circle) and right-click (draw a rectangle) on an image window.


In [ ]:
# Mouse callback function
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Draw a blue circle on left-click
        cv2.circle(img, (x, y), 10, (255, 0, 0), -1)
    elif event == cv2.EVENT_RBUTTONDOWN:
        # Draw a green rectangle on right-click
        cv2.rectangle(img, (x-10, y-10), (x+10, y+10), (0, 255, 0), 2)
    
    # Refresh the display
    cv2.imshow('Image', img)

# Create a blank image
img = np.zeros((512, 512, 3), np.uint8)

# Create a window and bind the mouse callback
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', mouse_callback)

# Display the image
cv2.imshow('Image', img)

# Wait for 'q' to quit
while True:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

## 6. Display Mouse Coordinates

This example displays the current mouse position (x, y coordinates) in the window title as you move the mouse over the image.



In [6]:

# Mouse callback function
def show_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        # Update the window title with coordinates
        cv2.setWindowTitle('Image', f'Coordinates: ({x}, {y})')
        # Display coordinates on the image
        img_copy = img.copy()  # Create a copy to avoid overwriting
        cv2.putText(img_copy, f'({x}, {y})', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Image', img_copy)

# Create a blank image (smaller size: 320x240)
img = np.zeros((240, 320, 3), np.uint8)

# Create a window and bind the mouse callback
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', show_coordinates)

# Display the image
cv2.imshow('Image', img)

# Main loop
while True:
    key = cv2.waitKey(10) & 0xFF  # Increased delay to 10ms
    if key == ord('q'):
        print("Q key pressed. Exiting...")
        break

cv2.destroyAllWindows()

Q key pressed. Exiting...


## 7. Draw Lines with Mouse Drag

This example allows you to draw lines by clicking and dragging the mouse (left button down to start, move to draw, release to finish).


In [3]:

# Global variables
drawing = False  # True if mouse is pressed
ix, iy = -1, -1  # Starting coordinates

# Mouse callback function
def draw_line(event, x, y, flags, param):
    global ix, iy, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y  # Record starting point

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            # Draw line from previous point to current
            cv2.line(img, (ix, iy), (x, y), (0, 255, 0), 2)
            ix, iy = x, y  # Update starting point for next segment

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        # Complete the line
        cv2.line(img, (ix, iy), (x, y), (0, 255, 0), 2)

# Create a blank image
img = np.zeros((512, 512, 3), np.uint8)

# Create a window and bind the mouse callback
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', draw_line)

while True:
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()



## 8. Crop Image Region with Mouse Selection

This example lets you select a rectangular region by dragging the mouse, then crops and displays the selected area in a new window.


In [7]:

# Global variables
cropping = False
x_start, y_start, x_end, y_end = -1, -1, -1, -1
image_copy = None

# Mouse callback function
def crop_image(event, x, y, flags, param):
    global x_start, y_start, x_end, y_end, cropping, image_copy

    if event == cv2.EVENT_LBUTTONDOWN:
        x_start, y_start = x, y
        cropping = True

    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping:
            image_copy = img.copy()
            cv2.rectangle(image_copy, (x_start, y_start), (x, y), (0, 255, 0), 2)
            cv2.imshow('Image', image_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        x_end, y_end = x, y
        cropping = False
        # Draw final rectangle
        cv2.rectangle(img, (x_start, y_start), (x_end, y_end), (0, 255, 0), 2)
        cv2.imshow('Image', img)

        # Crop the region if valid
        if x_start != x_end and y_start != y_end:
            cropped = img[min(y_start, y_end):max(y_start, y_end), min(x_start, x_end):max(x_start, x_end)]
            cv2.imshow('Cropped Region', cropped)

# Load an image and resize to smaller size (320x240)
img = cv2.imread('Sky.jpg')
if img is None:
    print("Error: Could not load image.")
    exit()
img = cv2.resize(img, (320, 240))

# Create a window and bind the mouse callback
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', crop_image)

# Main loop
while True:
    cv2.imshow('Image', img)
    key = cv2.waitKey(10) & 0xFF  # Increased delay to 10ms
    if key == ord('q'):
        print("Q key pressed. Exiting...")
        break

cv2.destroyAllWindows()

Q key pressed. Exiting...


## 9. Color Picker on Image

This example allows you to click on an image to get the BGR color value at that point and display it.


In [8]:

# Mouse callback function
def pick_color(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        # Get BGR values
        b, g, r = img[y, x]
        print(f"Color at ({x}, {y}): B={b}, G={g}, R={r}")
        
        # Display a small rectangle with the picked color
        color_img = np.zeros((100, 100, 3), np.uint8)
        color_img[:] = [b, g, r]
        cv2.imshow('Picked Color', color_img)

# Load an image and resize to smaller size (320x240)
img = cv2.imread('Sky.jpg')
if img is None:
    print("Error: Could not load image.")
    exit()
img = cv2.resize(img, (320, 240))

# Create a window and bind the mouse callback
cv2.namedWindow('Image')
cv2.setMouseCallback('Image', pick_color)

# Main loop
while True:
    cv2.imshow('Image', img)
    key = cv2.waitKey(10) & 0xFF  # Increased delay to 10ms
    if key == ord('q'):
        print("Q key pressed. Exiting...")
        break

cv2.destroyAllWindows()

Color at (86, 68): B=40, G=21, R=0
Color at (195, 91): B=71, G=57, R=29
Color at (198, 116): B=102, G=80, R=52
Color at (98, 178): B=71, G=44, R=8
Color at (254, 228): B=1, G=2, R=0
Color at (197, 115): B=119, G=94, R=62
Q key pressed. Exiting...


## 10. Interactive Free Drawing with Modes

This advanced example creates a simple drawing app where you can switch modes: free drawing (drag to draw), erase (right-click drag), and clear canvas (double-click).


In [5]:

# Global variables
drawing = False
erasing = False
mode = 'draw'  # 'draw' or 'erase'

# Mouse callback function
def interactive_draw(event, x, y, flags, param):
    global drawing, erasing

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        cv2.circle(img, (x, y), 5, (0, 0, 255), -1)  # Start drawing in green

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
        elif erasing:
            cv2.circle(img, (x, y), 10, (0, 0, 0), -1)  # Erase with black

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

    elif event == cv2.EVENT_RBUTTONDOWN:
        erasing = True
        cv2.circle(img, (x, y), 10, (0, 0, 0), -1)

    elif event == cv2.EVENT_RBUTTONUP:
        erasing = False

    elif event == cv2.EVENT_LBUTTONDBLCLK:
        # Clear the canvas
        img[:] = 0

# Create a blank image
img = np.zeros((512, 512, 3), np.uint8)

# Create a window and bind the mouse callback
cv2.namedWindow('Drawing App')
cv2.setMouseCallback('Drawing App', interactive_draw)

while True:
    cv2.imshow('Drawing App', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
